In [ ]:
import keras
import keras.layers as layers
from sklearn.metrics import classification_report
import pickle
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

from keras.layers import Dropout
from google.colab import files
import tensorflow as tf
from keras.callbacks import EarlyStopping
import numpy as np
tf.test.gpu_device_name()
from sklearn.metrics import confusion_matrix
from itertools import chain
from google.colab import drive
from collections import Counter
import os

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/canada/RR_29_09.zip'

Archive:  /content/drive/My Drive/canada/RR_29_09.zip
replace 1499_rr_5.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 1499_rr_5.pickle        
replace 32_rr_5.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 32_rr_5.pickle          
replace 1499_rr_3.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 1499_rr_3.pickle        
replace 32_rr_3.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 32_rr_3.pickle          
replace 1499_rr_2.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 1499_rr_2.pickle        
replace 32_rr_2.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 32_rr_2.pickle          
replace 1499_rr_1.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 1499_rr_1.pickle        
replace 32_rr_1.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 32_rr_1.pickle          


In [ ]:
#for diffrent classification this has to be implemented diffrently
def test_train_ott(fname="", train_out="", test_out=""):
    #otts = ['facebook', 'netflix', 'vimeo', 'hangouts', 'skype', 'youtube']
    otts = ['FaceBook', 'Netflix', 'Instagram', 'OtherOTT', 'YouTube']
    with open(fname, 'rb') as handle:        
        b = pickle.load(handle)
        np.random.shuffle(b)
        _list = list(chain.from_iterable(
            (_dict['videolabel'], _dict['ottlabel']) for _dict in b))
   
        c = Counter(_list)
        print('data: ', c) # for debug
        train, test = [], []
        
        #_lists ={"facebook": [], "netflix": [], "vimeo": [], "hangouts" : [], 
        #        'skype': [] , 'youtube': [] }

        _lists = {"FaceBook": [], "Netflix": [], "Instagram": [],
                   "OtherOTT" : [], 'YouTube': [] }

        for curr_dict in b:
             for ott in otts:
              if curr_dict['ottlabel'] == ott and curr_dict['videolabel'] == 'video':
                _lists[ott].append(curr_dict)
        for _list in _lists.values():
          train.extend(_list[:int(len(_list)*0.7)])
          test.extend(_list[int(len(_list)*0.7):])
    print('train: ', Counter(list(chain.from_iterable(
            (_dict['videolabel'], _dict['ottlabel']) for _dict in train))))
    print('test: ', Counter(list(chain.from_iterable(
            (_dict['videolabel'], _dict['ottlabel']) for _dict in test))))
    
    with open(train_out, 'wb') as handle:
      pickle.dump(train, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(test_out, 'wb') as handle:
        pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def test_train(fname="", train_out="", test_out=""):

    with open(fname, 'rb') as handle:        
        b = pickle.load(handle)
        np.random.shuffle(b)
        _list = list(chain.from_iterable(
            (_dict['videolabel'], _dict['ottlabel']) for _dict in b))
        
        c = Counter(_list)
        _list = []
        print('data: ', c) # for debug
        train, test = [], []
        num_video = c['video']
        counters ={"train_v": 0, "test_v": 0, "test_nv": 0, "train_nv" : 0}

        for curr_dict in b:
            if curr_dict['videolabel'] == 'video':
                if counters["test_v"] < num_video*0.3:
                    test.append(curr_dict)
                    counters["test_v"] += 1
                elif counters["train_v"] < num_video*0.7:
                    train.append(curr_dict)
                    counters["train_v"] += 1
            elif counters["test_nv"] < num_video*0.3:
                test.append(curr_dict)
                counters["test_nv"] += 1
            elif counters["train_nv"] < num_video*0.7:
                train.append(curr_dict)
                counters["train_nv"] += 1
            if sum(counters.values()) >= num_video*2:
                break
                
    print('train: ', Counter(list(chain.from_iterable(
            (_dict['videolabel'], _dict['ottlabel']) for _dict in train))))
    print('test: ', Counter(list(chain.from_iterable(
            (_dict['videolabel'], _dict['ottlabel']) for _dict in test))))
    
    train_dict =  Counter(list(chain.from_iterable(
            (_dict['videolabel'], _dict['ottlabel']) for _dict in train)))
    
    test_dict = Counter(list(chain.from_iterable(
            (_dict['videolabel'], _dict['ottlabel']) for _dict in test)))
    
    with open(train_out, 'wb') as handle:
      pickle.dump(train, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(test_out, 'wb') as handle:
        pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#for diffrent classification this has to be implemented diffrently
def pickle_image_generator_32_video(input_path, bs, lb, mode="train", aug=None):
    with open(input_path, 'rb') as handle:
        b = pickle.load(handle)
        if mode != "eval":
          np.random.shuffle(b)
        it = iter(b)
        while True:
            # initialize our batches of images and labels
            features = []
            labels = []
            # keep looping until we reach our batch size
            while len(features) < bs:
                # attempt to read the next line of the CSV file
                try:
                    curr_dict = it.__next__()
                except StopIteration:
                    it = iter(b)
                    if mode == "eval":
                        break

                label = curr_dict['videolabel']

                arr = np.zeros(32 * 32, dtype="uint8")
                for key, value in curr_dict.items():
                    if 'label' not in str(key):
                        arr[key] = value

                arr = arr.reshape((32, 32))
                # update our corresponding batches lists
                features.append(arr.reshape(32, 32, 1))
                labels.append(label)
            labels = lb.transform(np.array(labels))
            yield (np.array(features), labels)

In [ ]:
#for diffrent classification this has to be implemented diffrently
def pickle_image_generator_32_ott(input_path, bs, lb, mode="train", aug=None):
    with open(input_path, 'rb') as handle:
        b = pickle.load(handle)
        if mode != "eval":
          np.random.shuffle(b)
        it = iter(b)
        while True:
            # initialize our batches of images and labels
            features = []
            labels = []
            # keep looping until we reach our batch size
            while len(features) < bs:
                # attempt to read the next line of the CSV file
                try:
                    curr_dict = it.__next__()
                except StopIteration:
                    it = iter(b)
                    if mode == "eval":
                        break

                label = curr_dict['ottlabel']

                arr = np.zeros(32 * 32, dtype="uint8")
                for key, value in curr_dict.items():
                    if 'label' not in str(key):
                        arr[key] = value

                arr = arr.reshape((32, 32))
                # update our corresponding batches lists
                features.append(arr.reshape(32, 32, 1))
                labels.append(label)
            labels = lb.transform(np.array(labels))
            yield (np.array(features), labels)

Label: Video

In [ ]:
TRAIN_PICKLE = "train.pickle"
TEST_PICKLE = "test.pickle"
#SOURCES = ["/content/32_rr_1.pickle", "/content/32_rr_2.pickle", 
#          "/content/32_rr_3.pickle", "/content/32_rr_5.pickle"]
#SOURCES = ["/content/drive/My Drive/PickleTest/data.pickle"]
SOURCES = ["/content/drive/My Drive/flash/video_ott_10s_window_true.pickle"]
results = []
_train_dict = {}
_test_dict = {}

for SOURCE in SOURCES:
  for i in range(0, 5):
    print(SOURCE)
    test_train(fname=SOURCE, train_out=TRAIN_PICKLE, test_out=TEST_PICKLE)
    labels = ['video', 'novideo']
    # initialize the paths to our training and testing CSV files
    testLabels = []

    # initialize the number of epochs to train for and batch size
    NUM_EPOCHS = 75
    BS = 128

    # initialize the total number of training and testing image
    NUM_TRAIN_IMAGES = 0
    NUM_TEST_IMAGES = 0

    # open the training CSV file, then initialize the unique set of class
    # labels in the dataset along with the testing labels
    with open(TRAIN_PICKLE, 'rb') as handle:
        b = pickle.load(handle)
        NUM_TRAIN_IMAGES = len(b)

    with open(TEST_PICKLE, 'rb') as handle:
        b = pickle.load(handle)
        NUM_TEST_IMAGES = len(b)
        for curr_dict in b:
            testLabels.append(curr_dict['videolabel'])

    print(testLabels[0])  
    lb = LabelEncoder()
    lb.fit(labels)
    testLabels = lb.transform(testLabels)
    print(testLabels[0])
    # initialize both the training and testing image generators
    trainGen = pickle_image_generator_32_video(TRAIN_PICKLE, BS, lb,
                                      mode="train", aug=None)
    testGen = pickle_image_generator_32_video(TEST_PICKLE, BS, lb,
                                     mode="train", aug=None)

    model = keras.Sequential()

     #Layer 1
    #Conv Layer 1
    model.add(layers.Conv2D(filters = 6, 
                     kernel_size = 5, 
                     strides = 1, 
                     activation = 'relu', 
                     input_shape = (32,32,1)))
    #Pooling layer 1
    model.add(layers.MaxPooling2D(pool_size = 2, strides = 2))
    #Layer 2
    #Conv Layer 2
    model.add(layers.Conv2D(filters = 16, 
                     kernel_size = 5,
                     strides = 1,
                     activation = 'relu',
                     input_shape = (14,14,6)))
    model.add(Dropout(0.25))
    #Pooling Layer 2
    model.add(layers.MaxPooling2D(pool_size = 2, strides = 2))
    #Flatten
    model.add(layers.Flatten())
    #Layer 3
    #Fully connected layer 1
    model.add(layers.Dense(units = 120, activation = 'relu'))
    #Layer 4
    #Fully connected layer 2
    model.add(layers.Dense(units = 84, activation = 'relu'))
    model.add(Dropout(0.5))
    #Layer 5
    #Output Layer
    model.add(layers.Dense(units = 1, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 20)
    H = model.fit_generator(
        trainGen,
        steps_per_epoch=NUM_TRAIN_IMAGES // BS,
        validation_data=testGen,
        validation_steps=NUM_TEST_IMAGES // BS,
        epochs=NUM_EPOCHS,
        callbacks = [es])
    model.save('mod.h5')

     # serialize model to JSON
    model_json = model.to_json()
    with open("model_video_10s.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model_video_10s_weights.h5")
    print("Saved model to disk")

    # re-initialize our testing data generator, this time for evaluating
    testGen = pickle_image_generator_32_video(TEST_PICKLE, BS, lb,
                                     mode="eval", aug=None)

    # make predictions on the testing images, finding the index of the
    # label with the corresponding largest predicted probability
    predIdxs = model.predict_generator(testGen,
                                       steps=(NUM_TEST_IMAGES // BS) + 1)
    # show a nicely formatted classification report
    print("[INFO] evaluating network...")
    print(classification_report(testLabels, (predIdxs > 0.5),
                                target_names=lb.classes_))
    report = classification_report(testLabels, (predIdxs > 0.5),
                                target_names=lb.classes_)
    matrix = confusion_matrix(testLabels, (predIdxs > 0.5))
    print(matrix)
    print([SOURCE, matrix, report])
    results.append([SOURCE, matrix, report])


/content/drive/My Drive/flash/video_ott_10s_window_true.pickle
data:  Counter({'novideo': 19194, 'video': 4938, 'netflix': 2130, 'FaceBook': 1085, 'YouTube': 1064, 'OtherOTT': 578, 'Instagram': 81})
train:  Counter({'novideo': 6914, 'video': 3455, 'netflix': 1477, 'FaceBook': 757, 'YouTube': 754, 'OtherOTT': 411, 'Instagram': 56})
test:  Counter({'novideo': 2964, 'video': 1482, 'netflix': 653, 'FaceBook': 328, 'YouTube': 310, 'OtherOTT': 166, 'Instagram': 25})
novideo
0
Epoch 1/75
54/54 [==============================] - 3s 53ms/step - loss: 0.4593 - acc: 0.8145 - val_loss: 0.2023 - val_acc: 0.9280
Epoch 2/75
54/54 [==============================] - 1s 15ms/step - loss: 0.2064 - acc: 0.9187 - val_loss: 0.1728 - val_acc: 0.9416
Epoch 3/75
54/54 [==============================] - 1s 15ms/step - loss: 0.1756 - acc: 0.9317 - val_loss: 0.1432 - val_acc: 0.9507
Epoch 4/75
54/54 [==============================] - 1s 14ms/step - loss: 0.1586 - acc: 0.9385 - val_loss: 0.1384 - val_acc: 0.9487
E

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(results)

[['/content/drive/My Drive/flash/video_ott_10s_window_true.pickle', array([[1446,   36],
       [  35, 1447]]), '              precision    recall  f1-score   support\n\n     novideo       0.98      0.98      0.98      1482\n       video       0.98      0.98      0.98      1482\n\n    accuracy                           0.98      2964\n   macro avg       0.98      0.98      0.98      2964\nweighted avg       0.98      0.98      0.98      2964\n'], ['/content/drive/My Drive/flash/video_ott_10s_window_true.pickle', array([[1442,   40],
       [  38, 1444]]), '              precision    recall  f1-score   support\n\n     novideo       0.97      0.97      0.97      1482\n       video       0.97      0.97      0.97      1482\n\n    accuracy                           0.97      2964\n   macro avg       0.97      0.97      0.97      2964\nweighted avg       0.97      0.97      0.97      2964\n'], ['/content/drive/My Drive/flash/video_ott_10s_window_true.pickle', array([[1451,   31],
       [  3

In [ ]:
with open("/content/32_rr_video_results.txt", "w") as res_file:
  for result in results:
    res_file.write(str(result[0]))
    res_file.write(str(result[1]))
    res_file.write(str(result[2]))
    res_file.write("\n")    
  

OTT

In [ ]:
TRAIN_PICKLE = "train.pickle"
TEST_PICKLE = "test.pickle"
# SOURCES = ["/content/32_rr_1.pickle", "/content/32_rr_2.pickle", 
#          "/content/32_rr_3.pickle", "/content/32_rr_5.pickle"]
#SOURCES = ["/content/drive/My Drive/PickleTest/data.pickle"]
SOURCES = ["/content/drive/My Drive/flash/video_ott_10s_window_true.pickle"]
results = []
for SOURCE in SOURCES:
  for i in range(0, 5):
    print(SOURCE)
    test_train_ott(fname=SOURCE, train_out=TRAIN_PICKLE, test_out=TEST_PICKLE)
    #labels = ['youtube', 'facebook', 'vimeo', 'netflix']
    labels = ['FaceBook', 'Instagram', 'OtherOTT', 'YouTube']
    # initialize the paths to our training and testing CSV files
    testLabels = []

    # initialize the number of epochs to train for and batch size
    NUM_EPOCHS = 75
    BS = 128

    # initialize the total number of training and testing image
    NUM_TRAIN_IMAGES = 0
    NUM_TEST_IMAGES = 0

    # open the training CSV file, then initialize the unique set of class
    # labels in the dataset along with the testing labels
    with open(TRAIN_PICKLE, 'rb') as handle:
      b = pickle.load(handle)
      NUM_TRAIN_IMAGES = len(b)

    with open(TEST_PICKLE, 'rb') as handle:
      b = pickle.load(handle)
      NUM_TEST_IMAGES = len(b)
      for curr_dict in b:
          testLabels.append(curr_dict['ottlabel'])

    print(testLabels[0])  
    lb = LabelBinarizer()
    lb.fit(labels)
    testLabels = lb.transform(testLabels)
    print(testLabels[0])
    # initialize both the training and testing image generators
    trainGen = pickle_image_generator_32_ott(TRAIN_PICKLE, BS, lb,
                                    mode="train", aug=None)
    testGen = pickle_image_generator_32_ott(TEST_PICKLE, BS, lb,
                                   mode="train", aug=None)
    model = keras.Sequential()

    #Layer 1
    #Conv Layer 1
    model.add(layers.Conv2D(filters = 6, 
                   kernel_size = 5, 
                   strides = 1, 
                   activation = 'relu', 
                   input_shape = (32,32,1)))
    #Pooling layer 1
    model.add(layers.MaxPooling2D(pool_size = 2, strides = 2))
    #Layer 2
    #Conv Layer 2
    model.add(layers.Conv2D(filters = 16, 
                   kernel_size = 5,
                   strides = 1,
                   activation = 'relu',
                   input_shape = (14,14,6)))
    model.add(Dropout(0.25))
    #Pooling Layer 2
    model.add(layers.MaxPooling2D(pool_size = 2, strides = 2))
    #Flatten
    model.add(layers.Flatten())
    #Layer 3
    #Fully connected layer 1
    model.add(layers.Dense(units = 120, activation = 'relu'))
    #Layer 4
    #Fully connected layer 2
    model.add(layers.Dense(units = 84, activation = 'relu'))
    #Layer 5
    model.add(Dropout(0.5))
    #Output Layer
    model.add(layers.Dense(units = len(lb.classes_), activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    model.compile(loss='categorical_crossentropy',
                optimizer=keras.optimizers.Adam(),
                metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 20)
    H = model.fit_generator(
      trainGen,
      steps_per_epoch=NUM_TRAIN_IMAGES // BS,
      validation_data=testGen,
      validation_steps=NUM_TEST_IMAGES // BS,
      epochs=NUM_EPOCHS,
      callbacks = [es])
    model.save('mod.h5')

     # serialize model to JSON
    model_json = model.to_json()
    with open("model_ott_10s.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model_ott_10s_weights.h5")
    print("Saved model to disk")

      # re-initialize our testing data generator, this time for evaluating
    testGen = pickle_image_generator_32_ott(TEST_PICKLE, BS, lb,
                                   mode="eval", aug=None)

    # make predictions on the testing images, finding the index of the
    # label with the corresponding largest predicted probability
    predIdxs = model.predict_generator(testGen,
                                     steps=(NUM_TEST_IMAGES // BS) + 1)


    predIdxs = np.argmax(predIdxs, axis=1)

    # show a nicely formatted classification report
    print("[INFO] evaluating network...")
    report = classification_report(testLabels.argmax(axis=1), predIdxs,
    target_names=lb.classes_)
    print(report)
    matrix = confusion_matrix(testLabels.argmax(axis=1), predIdxs)
    print(matrix)
    results.append([SOURCE, matrix, report])



/content/drive/My Drive/flash/video_ott_10s_window_true.pickle
data:  Counter({'novideo': 19194, 'video': 4938, 'netflix': 2130, 'FaceBook': 1085, 'YouTube': 1064, 'OtherOTT': 578, 'Instagram': 81})
train:  Counter({'video': 1963, 'FaceBook': 759, 'YouTube': 744, 'OtherOTT': 404, 'Instagram': 56})
test:  Counter({'video': 845, 'FaceBook': 326, 'YouTube': 320, 'OtherOTT': 174, 'Instagram': 25})
FaceBook
[1 0 0 0]
Epoch 1/75
15/15 [==============================] - 3s 175ms/step - loss: 1.1892 - acc: 0.5536 - val_loss: 0.5823 - val_acc: 0.8138
Epoch 2/75
15/15 [==============================] - 0s 16ms/step - loss: 0.5600 - acc: 0.7979 - val_loss: 0.3698 - val_acc: 0.8893
Epoch 3/75
15/15 [==============================] - 0s 16ms/step - loss: 0.3925 - acc: 0.8641 - val_loss: 0.2994 - val_acc: 0.9062
Epoch 4/75
15/15 [==============================] - 0s 18ms/step - loss: 0.2880 - acc: 0.9010 - val_loss: 0.2361 - val_acc: 0.9167
Epoch 5/75
15/15 [==============================] - 0s 14ms

In [ ]:
print(results)

[['/content/drive/My Drive/flash/video_ott_10s_window_true.pickle', array([[322,   0,   4,   0],
       [  0,  16,   9,   0],
       [  1,   3, 168,   2],
       [  2,   0,   1, 317]]), '              precision    recall  f1-score   support\n\n    FaceBook       0.99      0.99      0.99       326\n   Instagram       0.84      0.64      0.73        25\n    OtherOTT       0.92      0.97      0.94       174\n     YouTube       0.99      0.99      0.99       320\n\n    accuracy                           0.97       845\n   macro avg       0.94      0.90      0.91       845\nweighted avg       0.97      0.97      0.97       845\n'], ['/content/drive/My Drive/flash/video_ott_10s_window_true.pickle', array([[324,   0,   2,   0],
       [  0,  22,   3,   0],
       [  3,   0, 171,   0],
       [  0,   0,   0, 320]]), '              precision    recall  f1-score   support\n\n    FaceBook       0.99      0.99      0.99       326\n   Instagram       1.00      0.88      0.94        25\n    OtherOTT

In [ ]:
with open("/content/32_rr_ott_results.txt", "a") as res_file:
  for result in results:
    res_file.write(str(result[0]))
    res_file.write(str(result[1]))
    res_file.write(str(result[2]))
    res_file.write("\n")    